In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

from tqdm import tqdm
import time
import random
from copy import deepcopy

import pickle

In [2]:
import pandas as pd
import numpy as np

In [ ]:
pip install tqdm

# Scrape

## Get Attractions

In [ ]:
attraction_links = []

In [ ]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [ ]:
miss_links = []

In [ ]:
soup = BeautifulSoup(driver.page_source)

### Finding Links Version 1

In [ ]:
attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})

In [ ]:
for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])

### Finding Links Version 2

In [ ]:
soup = BeautifulSoup(driver.page_source)

In [ ]:
attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))

In [ ]:
# Will only work if there's the second type of link in the webpage

In [ ]:
for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))

In [ ]:
len(attraction_links)

### Finding Next Button

In [ ]:
for i in soup.find_all('a', href=True):
    if i.text == '\nNext\n':
        link = 'https://www.tripadvisor.com/'+i['href']
        driver.get(link)
        break

In [ ]:
for i in soup.find_all('a', href=True):
    if i.text == 'Next':
        link = 'https://www.tripadvisor.com/'+i['href']
        driver.get(link)
        break

### Code to run

In [ ]:
attraction_links = []
miss_links = []

##

driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

##

# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(12)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('athens_links.pkl')

##




In [ ]:
attraction_links = []
miss_links = []

In [ ]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [ ]:
# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(12)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

### Fix Attraction Links

In [ ]:
attraction_links = list(set(attraction_links))

In [ ]:
len(attraction_links)

In [ ]:
#creating a test copy
t = deepcopy(attraction_links)

In [ ]:
for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)

In [ ]:
len(attraction_links)

In [ ]:
fixed_links = []

In [ ]:
for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

In [ ]:
fixed_links

In [ ]:
df = pd.DataFrame(fixed_links)

In [ ]:
df.to_pickle('athens_links.pkl')

In [ ]:
#convert dataframe to list, remember to call on column(0)
athens_links = df[0].tolist()

In [3]:
df = pd.read_pickle('athens_links.pkl')

In [4]:
athens_links = df[0].tolist()

# Get Info From Attractions

In [31]:
athens_links[0]

'https://www.tripadvisor.com/Attraction_Review-g189400-d12642863-Reviews-Gate_of_Medrese-Athens_Attica.html'

In [ ]:
driver.get(athens_links[0])

In [34]:
driver.find_element_by_xpath('//*[@id="HEADING"]').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="HEADING"]"}
  (Session info: chrome=79.0.3945.130)


## Definitions

In [469]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="HEADING"]').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/div[2]/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/span').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/span').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/a[2]').text

#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_resp_ar_responsive_0"]/div').text

def reviews(driver):
    return 

In [485]:
def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/div[2]/div/span').text


In [486]:
find_no_things_to_do(driver)

'#6 of 1,263 things to do in New York City'

In [488]:
#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/span').text

In [490]:
find_sights_landmarks(driver)

'Sights & Landmarks, Bridges'

In [492]:
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text

In [493]:
find_address(driver)

'New York City, NY 10038'

In [494]:
#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/span').text

In [495]:
find_summary_overview(driver)

'4.5'

In [496]:
#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/a[2]').text

In [497]:
number_of_reviews(driver)

'24,804 reviews'

In [498]:
#Find comments
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_resp_ar_responsive_0"]/div').text

In [457]:
find_name(driver)

'Brooklyn Bridge'

In [476]:
def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[1]/div/div[2]/div/span').text

In [477]:
find_no_things_to_do(driver)

'#6 of 1,263 things to do in New York City'

In [456]:
comments(driver)

"11 - 20 of 13,229 reviews\nR G\nDublin, Ireland\n815203\nReviewed 3 days ago\nA must do\nThis bridge connects two New York City boroughs, Manhattan and Brooklyn you can amble across a dedicated pedestrian walkway over the East river above the traffic below while enjoying great views of both boroughs .\nDate of experience: December 2019\nThank R G\nkarag205\nFort William, United Kingdom\n1216\nReviewed 3 days ago\nvia mobile\nLoved it\nBrooklyn Bridge and DUMBO were two of the highlights of our trip. We walked over the bridge, grabbed a coffee in DUMBO and sat for a while before walking back over. Would recommend, the vibe in DUMBO was brilliant.\nDate of experience: January 2020\nThank karag205\nDarren Lee S\n156\nReviewed 3 days ago\nA great view from the bridge\nWe had a great day visiting the Bridge, the sun was shinning and the sky was blue. We only walked half way but loved every minute of it. You can buy great magnets and bits for a $1 here too so don't pay over the...More\nDate

In [315]:
comments(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="taplc_location_reviews_list_resp_ar_responsive_0"]/div"}
  (Session info: chrome=79.0.3945.130)


In [314]:
number_of_reviews(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="taplc_location_detail_reviews_card_0"]/div[2]/a[2]"}
  (Session info: chrome=79.0.3945.130)


## Secondary Definitions

In [171]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[1]/h1').text

'Greek Folk Art Museum (Museo Ellinikis Laikis Technis)'

In [173]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[1]/div/div/span').text

'#131 of 319 things to do in Athens'

In [177]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[3]/span/div').text

'Museums, Art Museums'

In [179]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[2]/div/div[2]').text

'"House on Panos Street - Man and Tools"\nThis is a fascinating little museum and only 3 Euros. It is on the street between the Acropolis and...Read more\nReviewed June 26, 2018\n817kathye\n"Nothing here."\nMusuem empty but still charged admission fee. Only thing of interest was the old bath house but not ...Read more\nReviewed April 2, 2017\npartial_eclipse'

In [181]:
driver.find_element_by_xpath('//*[@id="component_23"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text

'42 Reviews'

In [182]:
driver.find_element_by_xpath('//*[@id="component_22"]/div[3]/div').text

"Traveller rating\nExcellent 16\nVery Good 18\nAverage 5\nPoor 0\nTerrible 3\nTraveller type\nFamilies\nCouples\nSolo\nBusiness\nFriends\nTime of year\nMar-May\nJun-Aug\nSep-Nov\nDec-Feb\nLanguage\nAll languages\nEnglish(28)\nGreek(3)\nFrench(2)\nMore\nKathy E wrote a review Jun. 2018\nPortland, Oregon2,696 contributions148 helpful votes\nHouse on Panos Street - Man and Tools\nThis is a fascinating little museum and only 3 Euros. It is on the street between the Acropolis and the Ancient Agora. We were in need of a few minutes sitting down (and a clean restroom) and went into this little gem. It won't be a highlight of your trip, but it is…\nRead more\nDate of experience: May 2018\nHelpful\nShare\npartial_eclipse wrote a review Apr. 2017\n32 contributions7 helpful votes\nNothing here.\nMusuem empty but still charged admission fee. Only thing of interest was the old bath house but not worth paying to get in.\nRead more\nDate of experience: April 2017\nHelpful\nShare\nVanessa_Fou wrote a 

In [ ]:
driver.find_element_by_xpath('').text

In [ ]:
driver.find_element_by_xpath('').text

In [ ]:
driver.find_element_by_xpath('').text

In [ ]:
driver.find_element_by_xpath('').text

In [334]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/h1').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[1]/div/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[3]/span/div').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[2]/div/div[2]').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text
#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="component_19"]/div[3]/div').text

#click the next button

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
                
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False
    
# def morebutton(driver):
#     try: 
#         more_button = driver.find_element_by_class_name('ulBlueLinks')
#         return more_button.click()
#     except:
#         return False
    
    
            

# def reviews(driver):
#     return driver.find_element_by_css_selector('#component_21 > div > div > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_component--2Qflo.attractions-attraction-review-atf-AttractionReviewATFLayout__header_container--dAJtk > div > div > div.ui_column.is-12-tablet.is-10-mobile.attractions-attraction-review-header-attraction-review-header__order0--1bYeG.attractions-attraction-review-header-attraction-review-header__columns--2j94i > h1').text

In [335]:

driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text


'280 Reviews'

In [336]:
number_of_reviews(driver)

'280 Reviews'

In [337]:
find_summary_overview(driver)

"Every beer lover who visits Santorini must do a tasting at one of the best breweries I've ever experienced!"

In [339]:
comments(driver)

"Traveller rating\nExcellent 155\nVery Good 80\nAverage 26\nPoor 14\nTerrible 5\nTraveller type\nFamilies\nCouples\nSolo\nBusiness\nFriends\nTime of year\nMar-May\nJun-Aug\nSep-Nov\nDec-Feb\nLanguage\nAll languages\nEnglish(218)\nItalian(20)\nGerman(7)\nMore\nPopular mentions\nAll reviewscrazy donkeyyellow donkeyt shirtsfree tastingbeer tastingdonkey beerslove beermicro brewerynice beergreat breweryquick visitsmall batchexo goniaworth a stopbuy a bottlebus stoptasting areaour honeymoonmerchandiseipa\nAnn Shen wrote a review Nov. 2019\n17 contributions1 helpful vote\nSmall batch beers\nThe beers made at Santorini brewing company are quite interesting and unique. Our pours were generous and we learnt a lot about each of their beers, the best of which was definitey Slow Donkey. Worth a stop over and a beer palate cleanser in between all of the wine.\nRead more\nDate of experience: October 2019\nHelpful\nShare\nLLV280Z wrote a review Oct. 2019\nDaytona Beach, Florida75 contributions10 help

In [324]:
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False
    

In [210]:
def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break

In [326]:
comments(driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="component_22"]/div[3]/div"}
  (Session info: chrome=79.0.3945.130)


In [322]:
driver.find_element_by_xpath('//*[@id="component_19"]/div[3]/div/div[3]').text

"Manuela wrote a review Jan. 2020\nTurin, Italy13 contributions1 helpful vote\nFantastic articles and very professional staff\nI discovered Mati during a holiday in Santorini some years ago. Since from the beginning I thought that it was fantastic. Few weeks ago I've bought a Christmas present contacting Mati by email. Mrs Diana was very kind and very professional, she gave me excellent suggestions .…\nRead more\nDate of experience: January 2020\nHelpful\nShare\nResponse from Alex K, General Manager at MATI Art Gallery\nResponded 2 weeks ago\nGood afternoon, Thank you very much for kind words, we really appreciate your feedback! Thank you again for the purchase and of course if you need anything we are here to help! All the best, Alex Kypris\nRead more"

In [319]:
driver.find_element_by_xpath('//*[@id="component_22"]/div[3]/div')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="component_22"]/div[3]/div"}
  (Session info: chrome=79.0.3945.130)


In [209]:
next_button.click()

In [203]:
driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g').click()

## Third Set of Definitions (by class)

In [479]:
def find_name(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/h1').text

def find_no_things_to_do(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[1]/div/div/span').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[2]/div/ul/li[3]/span/div').text

# Find Address
def find_address(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_resp_attraction_header_ar_responsive_0"]/div/div[2]/div/div[1]/div/div').text


#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[2]/div/div[2]').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_xpath('//*[@id="component_20"]/div/div/div[1]/div[1]/div/div/div[1]/div/span[2]').text
#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_xpath('//*[@id="component_19"]/div[3]/div').text

#click the next button

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
                
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False
    
# def morebutton(driver):
#     try: 
#         more_button = driver.find_element_by_class_name('ulBlueLinks')
#         return more_button.click()
#     except:
#         return False
    
    
            

# def reviews(driver):
#     return driver.find_element_by_css_selector('#component_21 > div > div > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd > div.attractions-attraction-review-atf-AttractionReviewATFLayout__atf_component--2Qflo.attractions-attraction-review-atf-AttractionReviewATFLayout__header_container--dAJtk > div > div > div.ui_column.is-12-tablet.is-10-mobile.attractions-attraction-review-header-attraction-review-header__order0--1bYeG.attractions-attraction-review-header-attraction-review-header__columns--2j94i > h1').text

In [480]:
def find_name(driver):
    return driver.find_element_by_class_name('ui_header').text

def find_no_things_to_do(driver):
    return driver.find_element_by_class_name('attractions-attraction-review-header-attraction-review-header__popIndex--H1_nL').text

#Find sights and landmarks
def find_sights_landmarks(driver):
    return driver.find_element_by_class_name('attractions-attraction-review-header-AttractionLinks__detail--2-xvX').text

# Find Address
def find_address(driver):
    return driver.find_element_by_class_name('attractions-contact-card-ContactCard__contactRow--3Ih6v').text

def find_meta_data(driver):
    return driver.find_element_by_class_name('masthead_h1').text

#Find summary overview
def find_summary_overview(driver):
    return driver.find_element_by_class_name('attractions-review-highlights-ReviewHighlights__snippetBlock--3KvE9').text

#Find number of reviews
def number_of_reviews(driver):
    return driver.find_element_by_class_name('attractions-attraction-review-header-attraction-review-header__ratingContainer--1lMqm').text

#Find About
def about_box(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_location_detail_overview_resp_ar_responsive_0"]/div/div/div[2]').text

#Find The Popular Mentions in comments
def popular_mentions(driver):
    return driver.find_element_by_xpath('//*[@id="taplc_detail_filters_ar_responsive_0"]/div/div[2]/div[3]/div').text.split('\n')

#Find 10 Most recent comments 
def comments(driver):
    return driver.find_element_by_class_name('ppr_priv_community_content').text

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
def morebutton(driver):
    try:
        more_button = driver.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
        return more_button.click()
    except:
        return False

### Testing Selenium

In [ ]:
attractions= []

In [ ]:
attraction_features = {}
# add to dictionary
try:
    attraction_features['name'] = find_name(driver)
except:
    attraction_features['name'] = None

# add to dictionary
try:
    attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
except:
    attraction_features['no_things_to_do'] = None

# add to dictionary
try:
    attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
except:
    attraction_features['sights_landmarks'] = None

# add to dictionary
try:
    attraction_features['address'] = find_address(driver)
except:
    attraction_features['address'] = None

# add to dictionary
try:
    attraction_features['summary_overview'] = find_summary_overview(driver)
except:
    attraction_features['summary_overview'] = None

# add to dictionary
try:
    attraction_features['num_reviews'] = number_of_reviews(driver)
except:
    attraction_features['num_reviews'] = None

# add to dictionary
try:
    attraction_features['about'] = about_box(driver)
except:
    attraction_features['about'] = None

# add to dictionary
try:
    attraction_features['popular_mentions'] = popular_mentions(driver)
except:
    attraction_features['popular_mentions'] = None

    
try:
    more_button = driver.find_element_by_class_name('ulBlueLinks')
    more_button.click()
    
except:
    pass
    
# add comments to dictionary
driver.implicitly_wait(5)
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

    
    
try:
    for i in driver.find_elements_by_class_name('next'):
        if i.text=='Next':
            next_button = i
            next_button.click()
            break
    
#add to list of dictionaries
attractions.append(attraction_features)

sequence = [x/10 for x in range(10, 15)]
time.sleep(random.choice(sequence))

## Click More on Comments

In [ ]:
comments(driver)

In [ ]:
attractions

## Comments 

In [ ]:
more_button = driver.find_element_by_class_name('ulBlueLinks')

In [ ]:
more_button.click()

In [ ]:
driver.implicitly_wait(10)

## Click Next Comments

In [8]:
for i in driver.find_elements_by_class_name('next'):
    if i.text=='Next':
        next_button = i
        next_button.click()
        break

NameError: name 'driver' is not defined

## For Loop Comments

In [ ]:
attraction_features['comments']= comments(driver)

In [ ]:
attraction_features['comments']+=(comments(driver))

In [ ]:
attraction_features['comments']+=''

In [ ]:
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None
    
    
    
    
# last_comment == comments(driver)

last_comment = comments(driver)

for i in range(3):    
    #click next
    if driver.find_element_by_class_name('next') == 'Next':
        next_button = driver.find_element_by_class_name('next')
        next_button.click()
    else:
        break

    #click more
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))
    except:
        pass

    #check if this is the same as last time
    if last_comment == comments(driver):
        break
    else:
        try:
            attraction_features['comments']+=(comments(driver))
        except:
            attraction_features['comments']+=''

        last_comment = comments(driver)

In [ ]:
# first click more
# then get comments!


sequence = [x/10 for x in range(15, 30)]

#loop through 1st 3 pages if they have a next
for next_comments in range(4):
    if driver.find_element_by_class_name('next')== None:
        break
    else:
        for i in driver.find_elements_by_class_name('next'):
            if i.text=='Next':
                next_button = i
                next_button.click()

    time.sleep(random.choice(sequence))

    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))

    except:
        pass


    #get comments
    # add to dictionary

    driver.implicitly_wait(5)
    try:
        attraction_features['comments']+=(comments(driver))
    except:
        attraction_features['comments']+=''

    time.sleep(random.choice(sequence))

## Selenium Code To Scrape

In [139]:
sequence = [x/10 for x in range(20, 35)]

In [ ]:
attractions = []
test = athens_links[5:]
len(athens_links)
len(test)

In [238]:
attractions = []

In [258]:
test = athens_links[5:]

In [259]:
len(test)

392

In [255]:
len(athens_links)

397

In [201]:
driver.get(test[0])

In [244]:
number_of_reviews(driver)

'1 Review'

In [166]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
# driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [197]:
morebutton(driver)

False

In [260]:
for i in tqdm(test):
    driver.get(i)
    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))

    
    attraction_features = {}
    
    attraction_features['link'] = i
    
    # add to dictionary
    try:
        attraction_features['name'] = find_name(driver)
    except:
        attraction_features['name'] = None

    # add to dictionary
    try:
        attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
    except:
        attraction_features['no_things_to_do'] = None

    # add to dictionary
    try:
        attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
    except:
        attraction_features['sights_landmarks'] = None

    # add to dictionary
    try:
        attraction_features['address'] = find_address(driver)
    except:
        attraction_features['address'] = None

    # add to dictionary
    try:
        attraction_features['summary_overview'] = find_summary_overview(driver)
    except:
        attraction_features['summary_overview'] = None

    # add to dictionary
    try:
        attraction_features['num_reviews'] = number_of_reviews(driver)
    except:
        attraction_features['num_reviews'] = None

    # add to dictionary
    try:
        attraction_features['about'] = about_box(driver)
    except:
        attraction_features['about'] = None

    # add to dictionary
    try:
        attraction_features['popular_mentions'] = popular_mentions(driver)
    except:
        attraction_features['popular_mentions'] = None
    
    
    try:
        morebutton(driver)
        time.sleep(random.choice(sequence)/2)
    except:
        pass
    
    time.sleep(random.choice(sequence))
    
    # add comments to dictionary
    
    try:
        attraction_features['comments'] = comments(driver)
    except:
        attraction_features['comments'] = None
    
    
    # last_comment == comments(driver)
    try:
        first_comment = comments(driver)
    except:
        pass
    
    

    
    #load more comments if there is any
    for i in (range(3)):

        try:
            if nextbutton(driver) == False:
                break
            else:
                nextbutton(driver)
                time.sleep(random.choice(sequence))

                try:
                    morebutton(driver)
                except:
                    pass

                time.sleep(random.choice(sequence))

                try:
                    if first_comment == comments(driver):
                        break
                    else:
                        first_comment = comments(driver)
                        
                except:
                    pass
                
                try:
                    attraction_features['comments']+=(comments(driver))
                except:
                    attraction_features['comments']+=''    
                    
                
        except:
            break
        
        
        try:
            #checks to make sure we're not recopying comments
            if last_comment == comments(driver):
                break
        except:
            pass
    #add to list of dictionaries
    attractions.append(attraction_features)






  0%|          | 0/392 [00:00<?, ?it/s]


  0%|          | 1/392 [00:15<1:41:24, 15.56s/it]


  1%|          | 2/392 [00:31<1:42:31, 15.77s/it]


  1%|          | 3/392 [00:51<1:50:17, 17.01s/it]


  1%|          | 4/392 [01:09<1:51:23, 17.23s/it]


  1%|▏         | 5/392 [01:39<2:16:26, 21.15s/it]


  2%|▏         | 6/392 [02:06<2:27:34, 22.94s/it]


  2%|▏         | 7/392 [02:28<2:24:29, 22.52s/it]


  2%|▏         | 8/392 [02:59<2:41:18, 25.21s/it]


  2%|▏         | 9/392 [03:15<2:22:40, 22.35s/it]


  3%|▎         | 10/392 [03:33<2:14:37, 21.15s/it]


  3%|▎         | 11/392 [03:49<2:04:21, 19.59s/it]


  3%|▎         | 12/392 [04:09<2:03:14, 19.46s/it]


  3%|▎         | 13/392 [04:27<2:00:15, 19.04s/it]


  4%|▎         | 14/392 [04:44<1:56:20, 18.47s/it]


  4%|▍         | 15/392 [05:02<1:56:11, 18.49s/it]


  4%|▍         | 16/392 [05:22<1:57:23, 18.73s/it]


  4%|▍         | 17/392 [05:53<2:20:13, 22.43s/it]


  5%|▍         | 18/392 [06:12<2:14:22, 21.56s/it]


  5%|▍    

 66%|██████▋   | 260/392 [1:27:19<54:07, 24.60s/it]


 67%|██████▋   | 261/392 [1:27:50<57:29, 26.34s/it]


 67%|██████▋   | 262/392 [1:28:20<59:39, 27.53s/it]


 67%|██████▋   | 263/392 [1:28:35<51:23, 23.91s/it]


 67%|██████▋   | 264/392 [1:28:52<46:32, 21.82s/it]


 68%|██████▊   | 265/392 [1:29:13<45:18, 21.41s/it]


 68%|██████▊   | 266/392 [1:29:33<43:56, 20.92s/it]


 68%|██████▊   | 267/392 [1:29:49<40:47, 19.58s/it]


 68%|██████▊   | 268/392 [1:30:05<38:13, 18.49s/it]


 69%|██████▊   | 269/392 [1:30:21<36:04, 17.60s/it]


 69%|██████▉   | 270/392 [1:30:38<35:43, 17.57s/it]


 69%|██████▉   | 271/392 [1:31:10<44:03, 21.85s/it]


 69%|██████▉   | 272/392 [1:31:41<49:04, 24.54s/it]


 70%|██████▉   | 273/392 [1:31:57<43:42, 22.04s/it]


 70%|██████▉   | 274/392 [1:32:15<40:45, 20.72s/it]


 70%|███████   | 275/392 [1:32:32<38:36, 19.80s/it]


 70%|███████   | 276/392 [1:33:04<45:04, 23.32s/it]


 71%|███████   | 277/392 [1:33:20<40:41, 21.23s/it]


 71%|███████   | 278/392 [1:

In [261]:
df = pd.DataFrame(attractions)

In [271]:
len(df['comments'][2])

8050

In [501]:
nextbutton(driver)

In [502]:
morebutton(driver)

False

In [272]:
df.to_pickle('athens_df.pkl')

In [231]:
df= df.sample(frac=1)

In [236]:
len(df['comments'][25])

2887

In [ ]:
attractions-attraction-review-header-attraction-review-header__reviewCount--3cEMP attractions-attraction-review-header-attraction-review-header__dotted_link--3p26D

In [9]:
from selenium.common.exceptions import NoSuchElementException

In [10]:
#click the next button

def nextbutton(driver):
    for j in driver.find_elements_by_class_name('next'):
        if j.text=='Next':
            next_button = j
            try:
                return next_button.click()
                break
            except:
                return False
                break
                
def morebutton(driver):
    try: 
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        return more_button.click()
    except:
        return False
            

In [503]:
def morebutton(driver):
    try: 
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        return more_button.click()
    except:
        return False

In [515]:
def morebutton(driver):
    try: 
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        return more_button.click()
    except:
        return False

In [508]:
more_button = driver.find_element_by_class_name('ulBlueLinks')

In [514]:
driver.find_element_by_class_name('ulBlueLinks').text

'More'

In [518]:
morebutton(driver)

In [517]:
nextbutton(driver)

In [ ]:
try:
    morebutton(driver)
except:
    print('pass')
    pass

In [ ]:
for i in range(3):
    try:
        

In [ ]:
attraction_features

In [ ]:
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

In [ ]:
for i in (range(3)):
    try:
        if nextbutton(driver) != False:
            nextbutton(driver)
            time.sleep(random.choice(sequence))
            
            try:
                morebutton(driver)
            except:
                pass
            
            time.sleep(random.choice(sequence))
            
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''
            
            
            
        else:
            break
        
    except:
        break
    
    
            

#         #click more
#         try:
#             more_button = driver.find_element_by_class_name('ulBlueLinks')
#             more_button.click()
#             time.sleep(random.choice(sequence))
#         except:
#             pass

#         #check if this is the same as last time
#         if last_comment == comments(driver):
# #             print('Last Comment is Current Comment')
#             break
#         else:
#             try:
#                 attraction_features['comments']+=(comments(driver))
#             except:
#                 attraction_features['comments']+=''

#             last_comment = comments(driver)

#             time.sleep(random.choice(sequence))

In [ ]:
    for i in tqdm(range(3)):    
        #click next
        for j in tqdm(driver.find_elements_by_class_name('next')):
            if j.text == 'Next':
                next_button = j
                try:
                    next_button.click()
                    time.sleep(random.choice(sequence))
                except:
                    pass

                break
            else:
                continue

        #click more
        try:
            more_button = driver.find_element_by_class_name('ulBlueLinks')
            more_button.click()
            time.sleep(random.choice(sequence))
        except:
            pass

        #check if this is the same as last time
        if last_comment == comments(driver):
            print('Last Comment is Current Comment')
            break
        else:
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''

            last_comment = comments(driver)

            time.sleep(random.choice(sequence))






In [ ]:
df= pd.DataFrame(attractions)

In [ ]:
df['comments'][0]

problem is if it doesn't have comments, it shouldnt go through the loop and waste time.

How to only get english? Should we get that?

That try statement is going to be to long. 

if there's a next on q and a and none on comments

In [ ]:
"""
1- If count > 3: press next
2- if there is a next button
3- Click more 
4- 


"""

In [ ]:
x = 3
for i in range(10):
    if x == 7:
        print(0)
        break
    else:
        print(x)
        x+=1
        

In [ ]:
count = 0
for i in range(3)

In [ ]:
"""
The problem is that we need a definitive reason to press next.
We want at most 3 comments.
If page is the same as last time. Break. 




"""

## Testing

In [ ]:
athens_links[0]

In [ ]:
driver.get(athens_links[6])

In [ ]:
# attraction_features['comments'] = None

In [ ]:
for i in driver.find_elements_by_class_name('next'):
                    print(i.text)
                    if i.text=='Next':
                        next_button = i
                        next_button.click()

In [ ]:
attraction_features = {}

In [ ]:
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

last_comment = comments(driver)

In [ ]:
sequence = [x/10 for x in range(10, 20)]

In [ ]:
for i in tqdm(range(3)):    
    #click next
    for j in tqdm(driver.find_elements_by_class_name('next')):
        if j.text == 'Next':
            next_button = j
            try:
                next_button.click()
                time.sleep(random.choice(sequence))
            except:
                pass
            
            break
        else:
            continue
    
    #click more
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))
    except:
        pass

    #check if this is the same as last time
    if last_comment == comments(driver):
        print('Last Comment is Current Comment')
        break
    else:
        try:
            attraction_features['comments']+=(comments(driver))
        except:
            attraction_features['comments']+=''

        last_comment = comments(driver)
        
        time.sleep(random.choice(sequence))
        
        
        
        
        

In [ ]:
len(attraction_features['comments'])

In [ ]:
attraction_features = {}

In [ ]:
    try:
        #loop through 1st 3 pages if they have a next
        for next_comments in range(4):
            if driver.find_element_by_class_name('next')== None:
                break
            else:
                for i in driver.find_elements_by_class_name('next'):
                    print(i.text)
                    if i.text=='Next':
                        next_button = i
                        next_button.click()
                        

            time.sleep(random.choice(sequence))

            try:
                more_button = driver.find_element_by_class_name('ulBlueLinks')
                more_button.click()
                time.sleep(random.choice(sequence))

            except:
                pass


            #get comments
            # add to dictionary

            driver.implicitly_wait(5)
            try:
                attraction_features['comments']+=(comments(driver))
            except:
                attraction_features['comments']+=''

            time.sleep(random.choice(sequence))
        
    except:
        pass

In [ ]:
attraction_features = {}
# add to dictionary
try:
    attraction_features['name'] = find_name(driver)
except:
    attraction_features['name'] = None

# add to dictionary
try:
    attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
except:
    attraction_features['no_things_to_do'] = None

# add to dictionary
try:
    attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
except:
    attraction_features['sights_landmarks'] = None

# add to dictionary
try:
    attraction_features['address'] = find_address(driver)
except:
    attraction_features['address'] = None

# add to dictionary
try:
    attraction_features['summary_overview'] = find_summary_overview(driver)
except:
    attraction_features['summary_overview'] = None

# add to dictionary
try:
    attraction_features['num_reviews'] = number_of_reviews(driver)
except:
    attraction_features['num_reviews'] = None

# add to dictionary
try:
    attraction_features['about'] = about_box(driver)
except:
    attraction_features['about'] = None

# add to dictionary
try:
    attraction_features['popular_mentions'] = popular_mentions(driver)
except:
    attraction_features['popular_mentions'] = None

    
try:
    more_button = driver.find_element_by_class_name('ulBlueLinks')
    more_button.click()
    
except:
    pass
    
# add comments to dictionary
driver.implicitly_wait(5)
try:
    attraction_features['comments'] = comments(driver)
except:
    attraction_features['comments'] = None

    
    
try:
    for i in driver.find_elements_by_class_name('next'):
        if i.text=='Next':
            next_button = i
            next_button.click()
            break
    
#add to list of dictionaries
attractions.append(attraction_features)

sequence = [x/10 for x in range(10, 15)]
time.sleep(random.choice(sequence))

In [ ]:
# first click more
# then get comments!


sequence = [x/10 for x in range(15, 30)]

#loop through 1st 3 pages if they have a next
for next_comments in range(4):
    if driver.find_element_by_class_name('next')== None:
        break
    else:
        for i in driver.find_elements_by_class_name('next'):
            if i.text=='Next':
                next_button = i
                next_button.click()
    
    time.sleep(random.choice(sequence))
    
    try:
        more_button = driver.find_element_by_class_name('ulBlueLinks')
        more_button.click()
        time.sleep(random.choice(sequence))

    except:
        pass
        
    
    #get comments
    # add to dictionary
    
    driver.implicitly_wait(5)
    try:
        attraction_features['comments']+=(comments(driver))
    except:
        attraction_features['comments']+=''
        
    time.sleep(random.choice(sequence))

# Santorini Activities

## Santorini Links

In [276]:
from tqdm import tnrange, tqdm_notebook as tqdm

In [284]:
attraction_links = []
miss_links = []

In [304]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette

In [286]:
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

In [287]:
for i in tqdm(range(4)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('santorini_links.pkl')

##

In [298]:
santorini_links = list(set(fixed_links))

In [283]:
attraction_links = []
miss_links = []

##

driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette
driver.get('https://www.tripadvisor.com/Attractions-g189400-Activities-Athens_Attica.html')

##

# Don't run attraction_links in same location
# attraction_links = []

#319 things to do in Athens
#Range= number of pages to go to
for i in tqdm(range(4)):
#if there is a next button, continue to click and scrape
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        print()
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
              
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    sequence = [x/10 for x in range(5, 10)]
    time.sleep(random.choice(sequence))

##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('santorini_links.pkl')

##

## Santorini Activities

In [405]:
attractions = []

In [299]:
len(santorini_links)

120

In [410]:
test = santorini_links[5:]

In [411]:
print(len(santorini_links), len(test))

120 115


In [412]:
# Scraper
#definitions will change
# log in first 
# then do a test run to get correct definitions

for i in tqdm(test):
    driver.get(i)
    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))

    
    attraction_features = {}
    
    attraction_features['link'] = i
    
    try:
        morebutton(driver)
        time.sleep(random.choice(sequence)/2)
    except:
        pass
    
    time.sleep(random.choice(sequence))
    
    
    
#     def find_meta_data(driver):
#     return driver.find_element_by_class_name('masthead_h1').text
    
    # add to dictionary
    try:
        attraction_features['meta'] = find_meta_data(driver)
    except:
        attraction_features['meta'] = None

    
    # add to dictionary
    try:
        attraction_features['name'] = find_name(driver)
    except:
        attraction_features['name'] = None

    # add to dictionary
    try:
        attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
    except:
        attraction_features['no_things_to_do'] = None

    # add to dictionary
    try:
        attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
    except:
        attraction_features['sights_landmarks'] = None

    # add to dictionary
    try:
        attraction_features['address'] = find_address(driver)
    except:
        attraction_features['address'] = None

    # add to dictionary
    try:
        attraction_features['summary_overview'] = find_summary_overview(driver)
    except:
        attraction_features['summary_overview'] = None

    # add to dictionary
    try:
        attraction_features['num_reviews'] = number_of_reviews(driver)
    except:
        attraction_features['num_reviews'] = None

    # add to dictionary
    try:
        attraction_features['about'] = about_box(driver)
    except:
        attraction_features['about'] = None

    # add to dictionary
    try:
        attraction_features['popular_mentions'] = popular_mentions(driver)
    except:
        attraction_features['popular_mentions'] = None
    
    
    # add comments to dictionary
    
    try:
        attraction_features['comments'] = comments(driver)
    except:
        attraction_features['comments'] = None
    
    
    # last_comment == comments(driver)
    try:
        first_comment = comments(driver)
    except:
        pass
    
    

    
    #load more comments if there is any
    for i in (range(3)):

        try:
            if nextbutton(driver) == False:
                break
            else:
                nextbutton(driver)
                time.sleep(random.choice(sequence))

                try:
                    morebutton(driver)
                    time.sleep(random.choice(sequence)/3)
                except:
                    pass

                time.sleep(random.choice(sequence))

                try:
                    if first_comment == comments(driver):
                        break
                    else:
                        first_comment = comments(driver)
                        
                except:
                    pass
                
                try:
                    attraction_features['comments']+=(comments(driver))
                except:
                    attraction_features['comments']+=''    
                    
                
        except:
            break
        
        
        try:
            #checks to make sure we're not recopying comments
            if last_comment == comments(driver):
                break
        except:
            pass
    #add to list of dictionaries
    attractions.append(attraction_features)



In [414]:
df = pd.DataFrame(attractions)

In [415]:
df.to_pickle('santorini_df.pkl')

# New York

In [450]:
driver = webdriver.Chrome() #browser should be named driver in the future, common coding etiquette

In [418]:
attraction_links = []
miss_links = []

In [424]:
17*30

510

In [430]:
attraction_links = list(set(attraction_links))

In [431]:
len(attraction_links)

480

In [423]:
sequence = [x/10 for x in range(9, 20)]

In [585]:
duplicate_attraction_links = deepcopy(attraction_links)

In [595]:
len(attraction_links)

480

In [596]:
480/30

16.0

In [607]:
for i in duplicate_attraction_links:
    if "Riverbank" in i:
        print(i)

In [605]:
nextbutton(driver)

In [587]:
len(duplicate_attraction_links)

480

In [616]:
len(attraction_links)

1672

In [612]:
attraction_links= list(set(attraction_links))

In [617]:
#dups is everything we've already done
#attraction links is new
#if in dupes, remove from attraction links
for i in duplicate_attraction_links:
    if i in attraction_links:
        attraction_links.remove(i)

In [615]:
count

480

In [618]:
1672-480

1192

In [619]:
len(attraction_links)

1192

In [608]:
sequence

[0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]

In [609]:
for i in tqdm(range(20)):
#if there is a next button, continue to click and scrape
    time.sleep(random.choice(sequence))
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #finds the attractions for each page
        attractions = soup.find_all('li',{'class':'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        
        #find all links in attraction
        for i in attractions:
            attraction_links.append(i.find('a', href=True)['href'])
    except: 
        pass
    
    try:
        soup = BeautifulSoup(driver.page_source)
        #another way to find
        attractions2 = (soup.find_all('div', {'class':'attraction_element_tall'}))
        
        for i in attractions2:
            for j in i.find_all('a', href=True):
                if j['href'][:18]== '/Attraction_Review':
                    attraction_links.append((j['href']))
    except:
        pass
            
    #find next button
    try:
        for i in soup.find_all('a', href=True):
            if i.text == '\nNext\n':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        pass
    
    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))
    
    try:
        for i in soup.find_all('a', href=True):
            if i.text == 'Next':
                link = 'https://www.tripadvisor.com/'+i['href']
                driver.get(link)
                break
    except:
        miss_links.append(i)
        sequence = [x/10 for x in range(5, 10)]
        time.sleep(random.choice(sequence))
        continue


    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))

In [622]:
##

for i in attraction_links:
    if i[-8:] == '#REVIEWS':
        attraction_links.remove(i)

attraction_links = list(set(attraction_links))        
        
##

fixed_links = []

##

for i in attraction_links:
    fixed_links.append('https://www.tripadvisor.com'+i)

##

df = pd.DataFrame(fixed_links)

##

df.to_pickle('new_york_list_pt2.pkl')

##

In [623]:
df

,0
0,https://www.tripadvisor.com/Attraction_Review-...
1,https://www.tripadvisor.com/Attraction_Review-...
2,https://www.tripadvisor.com/Attraction_Review-...
3,https://www.tripadvisor.com/Attraction_Review-...
4,https://www.tripadvisor.com/Attraction_Review-...
...,...
793,https://www.tripadvisor.com/Attraction_Review-...
794,https://www.tripadvisor.com/Attraction_Review-...
795,https://www.tripadvisor.com/Attraction_Review-...
796,https://www.tripadvisor.com/Attraction_Review-...


In [624]:
37-16

21

In [625]:
21*30

630

In [433]:
len(fixed_links)


480

In [626]:
fixed_links

['https://www.tripadvisor.com/Attraction_Review-g60763-d548351-Reviews-AXA_Gallery-New_York_City_New_York.html#REVIEWS',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d3784131-Reviews-Ebenezer_Gospel_Tabernacle-New_York_City_New_York.html#REVIEWS',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d18187881-Reviews-Saint_Ignatius_of_Antioch_Episcopal_Church-New_York_City_New_York.html#REVIEWS',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d4083896-Reviews-Kirk_Theatre-New_York_City_New_York.html',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d3751140-Reviews-Anne_Frank_Center_for_Mutual_Respect-New_York_City_New_York.html',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d2092726-Reviews-Heller_Gallery-New_York_City_New_York.html',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d10020612-Reviews-Tribeca_Film_Festival-New_York_City_New_York.html#REVIEWS',
 'https://www.tripadvisor.com/Attraction_Review-g60763-d12449707-Reviews-Artists_Fle

In [420]:
len(fixed_links)

480

In [519]:
attractions = []

In [633]:
test = fixed_links[5:]

In [634]:
len(test)

793

In [635]:
# Scraper
#definitions will change
# log in first 
# then do a test run to get correct definitions

for i in tqdm(test):
    driver.get(i)
    time.sleep(random.choice(sequence))
    time.sleep(random.choice(sequence))

    
    attraction_features = {}
    
    attraction_features['link'] = i
    
    try:
        morebutton(driver)
        time.sleep(random.choice(sequence)/2)
    except:
        pass
    
    time.sleep(random.choice(sequence))
    
    
    
#     def find_meta_data(driver):
#     return driver.find_element_by_class_name('masthead_h1').text
    
    # add to dictionary
    try:
        attraction_features['meta'] = find_meta_data(driver)
    except:
        attraction_features['meta'] = None

    
    # add to dictionary
    try:
        attraction_features['name'] = find_name(driver)
    except:
        attraction_features['name'] = None

    # add to dictionary
    try:
        attraction_features['no_things_to_do'] = find_no_things_to_do(driver)
    except:
        attraction_features['no_things_to_do'] = None

    # add to dictionary
    try:
        attraction_features['sights_landmarks'] = find_sights_landmarks(driver)
    except:
        attraction_features['sights_landmarks'] = None

    # add to dictionary
    try:
        attraction_features['address'] = find_address(driver)
    except:
        attraction_features['address'] = None

    # add to dictionary
    try:
        attraction_features['summary_overview'] = find_summary_overview(driver)
    except:
        attraction_features['summary_overview'] = None

    # add to dictionary
    try:
        attraction_features['num_reviews'] = number_of_reviews(driver)
    except:
        attraction_features['num_reviews'] = None

    # add to dictionary
    try:
        attraction_features['about'] = about_box(driver)
    except:
        attraction_features['about'] = None

    # add to dictionary
    try:
        attraction_features['popular_mentions'] = popular_mentions(driver)
    except:
        attraction_features['popular_mentions'] = None
    
    
    # add comments to dictionary
    
    try:
        attraction_features['comments'] = comments(driver)
    except:
        attraction_features['comments'] = None
    
    
    # last_comment == comments(driver)
    try:
        first_comment = comments(driver)
    except:
        pass
    
    

    
    #load more comments if there is any
    for i in (range(3)):

        try:
            if nextbutton(driver) == False:
                break
            else:
                nextbutton(driver)
                time.sleep(random.choice(sequence))

                try:
                    morebutton(driver)
                    time.sleep(random.choice(sequence)/3)
                except:
                    pass

                time.sleep(random.choice(sequence))

                try:
                    if first_comment == comments(driver):
                        break
                    else:
                        first_comment = comments(driver)
                        
                except:
                    pass
                
                try:
                    attraction_features['comments']+=(comments(driver))
                except:
                    attraction_features['comments']+=''    
                    
                
        except:
            break
        
        
        try:
            #checks to make sure we're not recopying comments
            if last_comment == comments(driver):
                break
        except:
            pass
    #add to list of dictionaries
    attractions.append(attraction_features)



In [637]:
df = pd.DataFrame(attractions)

In [528]:
df = pd.DataFrame(attractions)

In [639]:
df

,link,meta,name,no_things_to_do,sights_landmarks,address,summary_overview,num_reviews,about,popular_mentions,comments
0,https://www.tripadvisor.com/Attraction_Review-...,"AXA Gallery, New York City: Address, AXA Galle...",AXA Gallery,"#864 of 1,263 things to do in New York City","Shopping, Museums, Art Galleries","787 7th Ave, New York City, NY 10019-6018",4.5,2 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,None,"Марина А\nMoscow, Russia\n18097\nReviewed Apri..."
1,https://www.tripadvisor.com/Attraction_Review-...,"Ebenezer Gospel Tabernacle, New York City: Add...",Ebenezer Gospel Tabernacle,"#676 of 1,263 things to do in New York City","Sights & Landmarks, Churches & Cathedrals","225 Malcolm X Blvd, New York City, NY 10027-6341",4.0,9 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,None,"ValA508\n3\nReviewed November 22, 2019\nvia mo..."
2,https://www.tripadvisor.com/Attraction_Review-...,"Saint Ignatius of Antioch Episcopal Church, Ne...",Saint Ignatius of Antioch Episcopal Church,"#1,017 of 1,263 things to do in New York City","Sights & Landmarks, Churches & Cathedrals, More","552 W End Ave, New York City, NY 10024-2707",3.0,1 review,About\nClosed today\nHours Today: Closed\nSee ...,None,"Andrew M\n1520929\nReviewed August 13, 2019\nI..."
3,https://www.tripadvisor.com/Attraction_Review-...,"Kirk Theatre, New York City: Address, Phone Nu...",Kirk Theatre,"#592 of 1,263 things to do in New York City","Concerts & Shows, Theatres","410 W 42nd St, New York City, NY 10036-6809",4.0,15 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,None,"1 - 10 of 13 reviews\nIBMSweathog\nTucson, Ari..."
4,https://www.tripadvisor.com/Attraction_Review-...,"Anne Frank Center for Mutual Respect, New York...",Anne Frank Center for Mutual Respect,"#498 of 1,263 things to do in New York City","Museums, Speciality Museums","1325 Avenue of the Americas, 28th Floor, New Y...",4.5,18 reviews,"About\nAnne Frank Center for Mutual Respect, t...",None,"1 - 10 of 13 reviews\nCassysj\nBrooklyn, New Y..."
...,...,...,...,...,...,...,...,...,...,...,...
793,https://www.tripadvisor.com/Attraction_Review-...,"Church of the Ascension, New York City: Hours,...",Church of the Ascension,"#606 of 1,263 things to do in New York City","Sights & Landmarks, Churches & Cathedrals","5th Avenue at 10th Street, 5th Avenue at West ...",4.5,7 reviews,"About\nAscension, an Episcopal church in Green...",None,"DanaLosAngeles\nLos Angeles, California\n75384..."
794,https://www.tripadvisor.com/Attraction_Review-...,"Luhring Augustine Gallery, New York City: Hour...",Luhring Augustine Gallery,"#925 of 1,263 things to do in New York City","Museums, Shopping, Art Galleries","531 W 24th St, New York City, NY 10011-1104",5.0,1 review,"About\nFeatures modern artists from Europe, Ja...",None,"smuuuf\nBaden, Switzerland\n9049\nReviewed Aug..."
795,https://www.tripadvisor.com/Attraction_Review-...,"St. John Nepomucene, New York City: Address, P...",St. John Nepomucene,"#819 of 1,263 things to do in New York City","Sights & Landmarks, Churches & Cathedrals","411 E 66th St, 1st Avenue, New York City, NY 1...",4.0,3 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,None,"D0cumenta\nPrague, Czech Republic\n16433\nRevi..."
796,https://www.tripadvisor.com/Attraction_Review-...,"Arcadia Earth, New York City: Address, Arcadia...",Arcadia Earth,"#371 of 1,263 things to do in New York City","Museums, Speciality Museums, Science Museums","718 Broadway, New York City, NY 10003-9500",4.0,6 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,None,"daanddo\nLehigh Valley, Pennsylvania\n596177\n..."


In [638]:
df.to_pickle('nyc_df2.pkl')

In [534]:
nyc_td = df['no_things_to_do'].values

In [537]:
nyc_td.sort()

In [550]:
nyc_td= nyc_td.tolist()

AttributeError: 'list' object has no attribute 'tolist'

In [554]:
nyc_td[0].split(' ', 1)[0]

'#1'

In [555]:
nyc_n = []

In [556]:
for i in nyc_td:
    nyc_n.append(i.split(' ', 1)[0])

In [559]:
nyc_nu = []

In [560]:
for i in nyc_n:
    nyc_nu.append((i[1:]))

In [562]:
nyc_nu.sort()

In [569]:
a = []

In [571]:
for i in nyc_nu:
    a.append(i.replace(',', ''))

In [575]:
b = []

In [576]:
for i in a:
    b.append(int(i))

In [578]:
b.sort()

In [580]:
a = range(0,500)

In [582]:
count = 0
for i in a:
    if i not in b:
        count+=1
    

In [583]:
count

70

In [579]:
b

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [574]:
a

['1',
 '1016',
 '1032',
 '1033',
 '1039',
 '1040',
 '1053',
 '1065',
 '1069',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '2',
 '20',
 '200',
 '201',
 '202',
 '203',
 '204',

In [567]:
nyc_nu

['1',
 '1,016',
 '1,032',
 '1,033',
 '1,039',
 '1,040',
 '1,053',
 '1,065',
 '1,069',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '2',
 '20',
 '200',
 '201',
 '202',
 '203',

In [542]:
nyc_td

['#1 of 1,263 things to do in New York City',
 '#1,016 of 1,263 things to do in New York City',
 '#1,032 of 1,263 things to do in New York City',
 '#1,033 of 1,263 things to do in New York City',
 '#1,039 of 1,263 things to do in New York City',
 '#1,040 of 1,263 things to do in New York City',
 '#1,053 of 1,263 things to do in New York City',
 '#1,065 of 1,263 things to do in New York City',
 '#1,069 of 1,263 things to do in New York City',
 '#10 of 1,263 things to do in New York City',
 '#100 of 1,263 things to do in New York City',
 '#101 of 1,263 things to do in New York City',
 '#102 of 1,263 things to do in New York City',
 '#103 of 1,263 things to do in New York City',
 '#104 of 1,263 things to do in New York City',
 '#105 of 1,263 things to do in New York City',
 '#106 of 1,263 things to do in New York City',
 '#107 of 1,263 things to do in New York City',
 '#108 of 1,263 things to do in New York City',
 '#109 of 1,263 things to do in New York City',
 '#11 of 1,263 things to do

# Join Dataframes

In [3]:
ls

README.txt                             nyc_london.csv
TripAdvisor_parse2.ipynb               nyc_london_df_before_ifidf.csv
Tripadvisor_parse.ipynb                nyc_london_df_before_vectorize.csv
Tripadvisor_scrape.ipynb               nyc_london_pop_mentions.csv
Untitled.ipynb                         nyc_shopping.csv
Untitled1.ipynb                        santorini_df.pkl
athens_df.pkl                          santorini_links.pkl
athens_links.pkl                       scraping_london.ipynb
filename                               scraping_nyc.ipynb
florida_things_to_do.csv               shopping_london.ipynb
florida_things_to_do.csv.csv           ta_bangkok.csv
hotel-recommendation/                  ta_nyc_list.csv
london.csv                             ta_rome_list.csv
london_5_comments.csv                  top_5_comments_cleaned.csv
london_paris.csv                       tripadvisor_5_comments.ipynb
london_shopping.csv                    tripadvisor_df_b4_LDA.csv
new_york_list_pt2.pk

In [4]:
nyc0 = pd.read_pickle('nyc_df.pkl')

In [5]:
nyc0

,link,meta,name,no_things_to_do,sights_landmarks,address,summary_overview,num_reviews,about,popular_mentions,comments
0,https://www.tripadvisor.com/Attraction_Review-...,"Channel Gardens, New York City: Address, Phone...",Channel Gardens,"#329 of 1,263 things to do in New York City","Nature & Parks, Gardens","600 5th Ave, Between 49th and 50th Sts., New Y...",4.0,44 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,"[All reviews, rockefeller center, plants, benc...","1 - 10 of 18 reviews\nO6378WFannag\nChicago, I..."
1,https://www.tripadvisor.com/Attraction_Review-...,"Statue of Confucius, New York City: Address, S...",Statue of Confucius,"#561 of 1,263 things to do in New York City","Sights & Landmarks, Monuments & Statues","Confucius Plaza, Bowery, New York City, NY",3.5,42 reviews,About\nLOCAL WEATHER\nPowered by Weather Under...,None,"1 - 10 of 23 reviews\nlynnc156\nNew York City,..."
2,https://www.tripadvisor.com/Attraction_Review-...,"New York Public Library, New York City: Hours,...",New York Public Library,"#8 of 1,263 things to do in New York City","Museums, Speciality Museums","476 5th Ave New York, NY 10018, New York City,...",4.5,"14,265 reviews",About\nTwo marble lions mark the entrance to t...,"[All reviews, beautiful building, reading room...","1 - 10 of 7,591 reviews\nRose193\n319\nReviewe..."
3,https://www.tripadvisor.com/Attraction_Review-...,"James A. Farley Post Office, New York City: Ad...",James A. Farley Post Office,"#284 of 1,263 things to do in New York City","Sights & Landmarks, Historic Sites","421 8th Ave, New York City, NY 10001",4.0,100 reviews,About\nA great-columned edifice; as much a pos...,None,"1 - 10 of 39 reviews\nBeachStone2C\nHelsinki, ..."
4,https://www.tripadvisor.com/Attraction_Review-...,"Brooklyn Bridge, New York City: Hours, Address...",Brooklyn Bridge,"#6 of 1,263 things to do in New York City","Sights & Landmarks, Bridges","New York City, NY 10038",4.5,"24,804 reviews","About\nOpened in 1883, the Brooklyn Bridge pro...","[All reviews, bike lane, great walk, new york,...","1 - 10 of 13,229 reviews\n988chrisa\nPaignton,..."
...,...,...,...,...,...,...,...,...,...,...,...
475,https://www.tripadvisor.com/Attraction_Review-...,"George M. Cohan Statue, New York City: Hours, ...",George M. Cohan Statue,"#523 of 1,263 things to do in New York City","Sights & Landmarks, Monuments & Statues","Father Duffy Square, New York City, NY",4.0,22 reviews,What travellers are saying\nLocated in Father ...,"[All reviews, times square, composer, cagney, ...","1 - 10 of 14 reviews\n7matts9\nStaffordshire, ..."
476,https://www.tripadvisor.com/Attraction_Review-...,"Lincoln Tunnel, New York City: Address, Lincol...",Lincoln Tunnel,"#497 of 1,263 things to do in New York City","Sights & Landmarks, Bridges","500 J F Kennedy Blvd E, New York City, NY",3.5,73 reviews,What travellers are saying\nIt was built many ...,"[All reviews, hudson river, new york, engineer...","1 - 10 of 46 reviews\ngaurav c\nNew Delhi, Ind..."
477,https://www.tripadvisor.com/Attraction_Review-...,"The Metropolitan Museum of Art, New York City:...",The Metropolitan Museum of Art,"#4 of 1,263 things to do in New York City","Museums, Sights & Landmarks, More","1000 5th Ave, New York City, NY 10028-0198",5.0,"54,411 reviews",About\nAt New York City's most visited museum ...,"[All reviews, american wing, temple of dendur,...","1 - 10 of 31,673 reviews\nMSUmindy\nShelby, Mi..."
478,https://www.tripadvisor.com/Attraction_Review-...,"Morris-Jumel Mansion, New York City: Address, ...",Morris-Jumel Mansion,"#238 of 1,263 things to do in New York City","Museums, Speciality Museums","65 Jumel Ter, Washington Heights, New York Cit...",4.0,98 reviews,"About\nManhattan's oldest house, somewhat of a...","[All reviews, aaron burr, ghost adventures, am...","1 - 10 of 79 reviews\n601eddies\nBronx, New Yo..."


In [6]:
nyc1= pd.read_pickle('nyc_df2.pkl')

In [7]:
nyc1.columns ==nyc0.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [10]:
nyc = [nyc1,nyc0]

In [11]:
nyc_df = pd.concat(nyc)

In [13]:
athens = pd.read_pickle('athens_df.pkl')

In [14]:
santorini = pd.read_pickle('santorini_df.pkl')

In [15]:
athens['meta'] = None

In [16]:
santorini.columns

Index(['link', 'meta', 'name', 'no_things_to_do', 'sights_landmarks',
       'address', 'summary_overview', 'num_reviews', 'about',
       'popular_mentions', 'comments'],
      dtype='object')

In [17]:
athens1= athens[['link', 'meta', 'name', 'no_things_to_do', 'sights_landmarks',
       'address', 'summary_overview', 'num_reviews', 'about',
       'popular_mentions', 'comments']]

In [18]:
athens1.columns == santorini.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [19]:
greece = pd.concat([athens1, santorini])

In [20]:
greece.columns == nyc_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [21]:
df = pd.concat([greece,nyc_df])

# NLP on Comments

In [29]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
# first of all, let us remove some useless words to see how our documents change
stop_words = set(stopwords.words('english')) 

In [27]:
#set index- will be needed later for calling the matrix
df.set_index('name', inplace=True)

In [50]:
# calculate cosine similarity between documents - we use tfidf
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [52]:
len(missing)

70

## Get rid of nan

In [65]:
df.index

Index([                                                             'Gate of Medrese',
       'Museum of the History of the Greek Costume of the Lyceum Club of Greek Women',
                                                              'Eleni Marneri Galerie',
                                                                    'Stoa Avissinias',
                                                            'Ikastikos Kyklos Sianti',
                                                          'ELEYThERH EKFRASH Theater',
                                                                                 None,
                                                 'Urban Rail Transport S.A - Tramway',
                                                             'Masjid Al-Salam Mosque',
                                                                 'Aristotle's Lyceum',
       ...
                                                                    'Algonquin Hotel',
                                

In [70]:
df = df[~df.index.isna()]

In [71]:
df

,link,meta,no_things_to_do,sights_landmarks,address,summary_overview,num_reviews,about,popular_mentions,comments
name,,,,,,,,,,
Gate of Medrese,https://www.tripadvisor.com/Attraction_Review-...,None,#225 of 319 things to do in Athens,"Sights & Landmarks, Points of Interest & Landm...",None,"""FORMER MADRASA ENTRY PORTAL""\nThis doorway is...",6 Reviews,None,None,Traveller rating\nExcellent 1\nVery Good 0\nAv...
Museum of the History of the Greek Costume of the Lyceum Club of Greek Women,https://www.tripadvisor.com/Attraction_Review-...,None,#284 of 319 things to do in Athens,"Museums, History Museums",None,None,5 Reviews,None,None,Traveller rating\nExcellent 0\nVery Good 1\nAv...
Eleni Marneri Galerie,https://www.tripadvisor.com/Attraction_Review-...,None,#100 of 319 things to do in Athens,"Shopping, Museums, Art Galleries",None,None,21 Reviews,None,None,Traveller rating\nExcellent 19\nVery Good 1\nA...
Stoa Avissinias,https://www.tripadvisor.com/Attraction_Review-...,None,#293 of 319 things to do in Athens,"Shopping, Museums, More",None,Stoa Avissinias is a reputable shop in the old...,None,None,None,
Ikastikos Kyklos Sianti,https://www.tripadvisor.com/Attraction_Review-...,None,#220 of 319 things to do in Athens,"Shopping, Museums, Art Galleries",None,"""Ikastikos Kiklos Sianti"" is an art space with...",1 Review,None,None,Traveller rating\nExcellent 1\nVery Good 0\nAv...
...,...,...,...,...,...,...,...,...,...,...
George M. Cohan Statue,https://www.tripadvisor.com/Attraction_Review-...,"George M. Cohan Statue, New York City: Hours, ...","#523 of 1,263 things to do in New York City","Sights & Landmarks, Monuments & Statues","Father Duffy Square, New York City, NY",4.0,22 reviews,What travellers are saying\nLocated in Father ...,"[All reviews, times square, composer, cagney, ...","1 - 10 of 14 reviews\n7matts9\nStaffordshire, ..."
Lincoln Tunnel,https://www.tripadvisor.com/Attraction_Review-...,"Lincoln Tunnel, New York City: Address, Lincol...","#497 of 1,263 things to do in New York City","Sights & Landmarks, Bridges","500 J F Kennedy Blvd E, New York City, NY",3.5,73 reviews,What travellers are saying\nIt was built many ...,"[All reviews, hudson river, new york, engineer...","1 - 10 of 46 reviews\ngaurav c\nNew Delhi, Ind..."
The Metropolitan Museum of Art,https://www.tripadvisor.com/Attraction_Review-...,"The Metropolitan Museum of Art, New York City:...","#4 of 1,263 things to do in New York City","Museums, Sights & Landmarks, More","1000 5th Ave, New York City, NY 10028-0198",5.0,"54,411 reviews",About\nAt New York City's most visited museum ...,"[All reviews, american wing, temple of dendur,...","1 - 10 of 31,673 reviews\nMSUmindy\nShelby, Mi..."


In [ ]:
#create the matrix
tfidf_matrix = tf.fit_transform(df['comments_real2'])